In [ ]:
from RPA.Windows import Windows
from time import sleep
from datetime import datetime
from RPA.Desktop import Desktop
import gspread as gs
from tasks.crear_factura_ER import * 
import gspread as gs
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

win = Windows()
desk = Desktop()
gc = gs.service_account(filename='tdpcorp-139fbe6dbc3c.json')

In [ ]:
# connection to sql server 
import pyodbc
db_name = 'SBO_TDPC_TEST'

## string connection
def get_conn_sql(db_name='SBO_TDPC_TEST'):
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER=TDPSERVERW19;DATABASE=%s;UID=sa;PWD=Passw0rd' % db_name)

    return conn

conn = get_conn_sql()

In [ ]:
def get_servision(conn, codigos):
    # return in df
    query = """SELECT *
     FROM [@SYP_SERVICIOS] T0 WHERE T0.[Code] IN (%s)""" % ','.join([str(f"'{c}'") for c in codigos])
    df = pd.read_sql(query, conn)
    return df
get_servision(conn, [9865611, 9865911])

In [ ]:
# leer datos desde google sheet

gc = gs.service_account(filename='tdpcorp-139fbe6dbc3c.json')
WB = 'https://docs.google.com/spreadsheets/d/11HyZN62kfRManwuUz5jNDj7wtr8YFWJz1M5zWlTcS9Q/edit#gid=0'

workbook = gc.open_by_url(WB)
worksheet = workbook.get_worksheet_by_id(0)
def cargar_data():
    # cargar en un dataframe
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    # completar datos en tipo blanco y na
    df['Tipo de Documento'] = df['Tipo de Documento'].replace('', '01').fillna('01')

    # actualizar tipo de compra solo si Numero CC/ER esta vacio
    df.loc[df['Numero CC/ER'] == '', 'Tipo de Compra'] = '01'
    df['Tipo de Compra'] = df['Tipo de Compra'].fillna('CC')

    # casting to datetime format dd.mm.yyyy
    df['Fecha de contabilización'] = pd.to_datetime(df['Fecha de contabilización'], format='%d.%m.%Y')
    df['Fecha del documento'] = pd.to_datetime(df['Fecha del documento'], format='%d.%m.%Y')
    df['Fecha de vencimiento'] = pd.to_datetime(df['Fecha de vencimiento'], format='%d.%m.%Y')
    # casting to float
    df['Precio por unidad'] = df['Precio por unidad'].astype(float)
    df['Total (ML)'] = df['Total (ML)'].astype(float)
    # add services name
    servicios_df = get_servision(conn, df['Codigo de Gasto'].unique())[['Code', 'U_SYP_Concepto']]
    df = df.merge(servicios_df, left_on='Codigo de Gasto', right_on='Code', how='left')
    # add index
    df['No.Ref.del acreedor'] = df['Tipo de Documento'].str.cat(df[['Serie del Documento','Correlativo del Documento']], sep='-')
    df.set_index('No.Ref.del acreedor', inplace=True)
    return df



In [ ]:
df = cargar_data()

In [ ]:
import requests
import json

URL_BASE = 'https://192.168.208.109:50000'

# URL para el inicio de sesión (login)
login_url = f'{URL_BASE}/b1s/v1/Login'

# Datos de inicio de sesión
credentials = {
    'CompanyDB': db_name,
    'UserName': 'manager',
    'Password': '1234'
}

def login(credentials):
    # Configurar encabezados de la solicitud
    headers = {
        'Content-Type': 'application/json'
    }
    json_data = json.dumps(credentials)
    # Realizar la solicitud POST para el inicio de sesión
    response = requests.post(login_url, data=json_data, headers=headers, verify=False)

    # Verificar la respuesta y obtener el token de autenticación
    session_id = response.json()['SessionId']
    return session_id


In [ ]:

def make_invoices(df):
    invoices = []
    new_df = df[df['DocNum'] == '']
    for index in new_df.index.unique():
        infoice_df = df.loc[[index]]
        invoice = {
            "CardCode": infoice_df['Proveedor'].iloc[0],
            "NumAtCard": index,
            # fechas
            "DocDate": infoice_df['Fecha de contabilización'].iloc[0].strftime('%Y-%m-%d'),
            "DocDueDate": infoice_df['Fecha de vencimiento'].iloc[0].strftime('%Y-%m-%d'),
            "TaxDate": infoice_df['Fecha del documento'].iloc[0].strftime('%Y-%m-%d'),
            "DocType": "S",
            # Numero de factura
            "U_SYP_MDTD": infoice_df['Tipo de Documento'].iloc[0],
            "U_SYP_MDSD": infoice_df['Serie del Documento'].iloc[0],
            "U_SYP_MDCD": infoice_df['Correlativo del Documento'].iloc[0],
            "U_SYP_TCOMPRA": infoice_df['Tipo de Compra'].iloc[0],
            "comments": infoice_df['Comentarios'].iloc[0],
        }
        if infoice_df['Tipo de Compra'].iloc[0] == 'CC':
            invoice['U_SYP_CODERCC'] = infoice_df['Numero CC/ER'].iloc[0]

        invoice_lines = []
        for index, row in infoice_df.iterrows():
            invoice_lines.append({
                "AccountCode": row['Codigo de Gasto'],
                "U_SYP_TIPOSERV": row['Codigo de Gasto'],
                "Description": row['U_SYP_Concepto'],
                "Quantity": 1,
                "UnitPrice": row['Precio por unidad'],
                "TaxCode": row['Indicador de impuestos'],
                # centro de costo
                "CostingCode": row['Centro de Costos'],
                "CostingCode2": row['Unidad de negocio'],
                "CostingCode3": row['Local'],
                "CostingCode4": row['Canal de distribución'],
                # Total (ML)
                "LineTotal": row['Total (ML)']
            })

        invoice['DocumentLines'] = invoice_lines
        invoices.append(invoice)
    return invoices



In [ ]:
def create_invoice(invoice, session_id):

    headers = {
        'Content-Type': 'application/json',
        'Cookie': f'B1SESSION={session_id}'
    }
    json_data = json.dumps(invoice)
    response = requests.post(f'{URL_BASE}/b1s/v1/PurchaseInvoices', data=json_data, headers=headers, verify=False)
    return response.json()



In [ ]:
from rich import print

invoices = make_invoices(df)
session_id = login(credentials)
#print(invoices)
for invoice in invoices:
    req = create_invoice(invoice, session_id)
    try:
        df.loc[req["NumAtCard"], "DocNum"] = str(req["DocNum"])
    except:
        df.loc[invoice["NumAtCard"], "Error"] = req["error"]["message"]["value"]
print(df[['DocNum', 'Error']].fillna('-'))

In [ ]:
import numpy as np
doc_num = df['DocNum'].fillna('').to_numpy().reshape(-1, 1)
worksheet.update('W2', doc_num.tolist())
errors = df['Error'].fillna('').to_numpy().reshape(-1, 1)
worksheet.update('X2', errors.tolist())